In [15]:
import requests
import os
import json
from dotenv import load_dotenv
import pathlib
import pandas as pd
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

# name the file we want
env_file_name = 'lrhf97_twit_cred.env'

# designate the path of the file
curr_file = pathlib.Path.cwd()

# join them
env_path = curr_file.joinpath(env_file_name)

# load the file
load_dotenv(env_path)

bearer_token = os.getenv("BEARER_TOKEN")
next_token=None


def create_url():
    # Replace with user ID below - RidgeSupply =3413455348
    user_id = 3413455348
    max_users = 100
    
    return "https://api.twitter.com/2/users/{}/followers?max_results={}".format(user_id,max_users)

def get_params():
    return {'user.fields': 'id,name,username,created_at,description,public_metrics,verified,location',
            'next_token': {}}
    # return {'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
    #         'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
    #         'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
    #         'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
    #         'next_token': {}}

def bearer_oauth(r):

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r


def connect_to_endpoint(url, params, next_token=None):
    params['next_token'] = next_token # param object recieved form the create_url function
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params,next_token)
    # print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response


ridge_followers= main()

200


In [16]:
df=pd.DataFrame.from_dict(ridge_followers['data'])
df

,name,verified,description,created_at,username,id,public_metrics,location
0,J,False,"Husband, dad, cyclist and a F'n 🦧",2015-07-21T20:02:19.000Z,DumbMoney_AMC,3386288339,"{'followers_count': 2, 'following_count': 72, ...",NaN
1,frankgiraffe,False,Living at the intersection of geekery and the ...,2008-04-14T16:56:54.000Z,frankgiraffe,14387509,"{'followers_count': 215, 'following_count': 37...","SF, CA USA"
2,Jordan Spoon,False,Slinging paints on cars for @maconisetupshop -...,2009-04-20T14:52:11.000Z,jspoon13,33535739,"{'followers_count': 85, 'following_count': 606...","Kernersville, NC"
3,f3morganton,False,"Fitness, Fellowship, & Faith coming to Morgant...",2021-07-22T18:55:59.000Z,f3morganton,1418283365577461764,"{'followers_count': 22, 'following_count': 78,...",NaN
4,DSRPT cycling,False,,2021-07-18T18:03:36.000Z,CyclingDsrpt,1416820910712135682,"{'followers_count': 0, 'following_count': 47, ...",NaN
...,...,...,...,...,...,...,...,...
95,Ken Ray,False,"Landscape architect, bicycle advocate, home br...",2009-03-20T02:24:59.000Z,kenallenray,25440153,"{'followers_count': 230, 'following_count': 56...",NaN
96,Mock Orange Bikes,False,,2011-01-24T21:44:45.000Z,mockorangebikes,242476589,"{'followers_count': 713, 'following_count': 61...","Winston-Salem, NC"
97,Lazer Sport North America,False,Lazer Sport North America\nBest Fit - Guarante...,2016-10-26T14:53:04.000Z,LazerSportNA,791291562940518400,"{'followers_count': 515, 'following_count': 27...",Irvine California USA
98,Chris Gasche,False,Husband to Carrie...Dad to Gabe...HOPE followe...,2018-03-05T01:06:42.000Z,ChrisGasche,970465593588625408,"{'followers_count': 16, 'following_count': 98,...",NaN


In [17]:
ridge_followers['meta']['next_token']

'MTN0C90S1GFHCZZZ'

In [18]:
import csv
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['id', 'created_at', 'location', 'description','username','name','verified', 'followers_count', 'following_count', 'tweet_count','listed_count'])
csvFile.close()

In [19]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. ID
        id = tweet['id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('location' in tweet):   
            location = tweet['location']
        else:
            location = " "

        # 4. description
        description = tweet['description']

        # # 5. Language
        username = tweet['username']
        name = tweet['name']
        verified = tweet['verified']


        # 6. Tweet metrics
        followers_count = tweet['public_metrics']['followers_count']
        following_count = tweet['public_metrics']['following_count']
        tweet_count = tweet['public_metrics']['tweet_count']
        listed_count = tweet['public_metrics']['listed_count']


        # # 7. source
        # source = tweet['source']

        # # 8. Tweet text
        # text = tweet['text']
        
        # Assemble all data in a list
        res = [id, created_at, location, description,username,name,verified, followers_count, following_count, tweet_count,listed_count]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Followers added from this response: ", counter) 

In [13]:
append_to_csv(ridge_followers,'data.csv')

# of Followers added from this response:  100


## This is for looping over tweets - going to try to convert it to get the total list of followers

In [20]:
#Inputs for followers
# bearer_token = bearer_oauth
# headers = create_headers(bearer_token)
# keyword = "xbox lang:en"
# start_list =    ['2021-01-01T00:00:00.000Z',
#                  '2021-02-01T00:00:00.000Z',
#                  '2021-03-01T00:00:00.000Z']

# end_list =      ['2021-01-31T00:00:00.000Z',
#                  '2021-02-28T00:00:00.000Z',
#                  '2021-03-31T00:00:00.000Z']




max_results = 500
max_users=100

#Total number of followers we collected from the loop
total_followers = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['id', 'created_at', 'location', 'description','username','name','verified', 'followers_count', 'following_count', 'tweet_count','listed_count'])
csvFile.close()
params['next_token']=None

for i in range(0,int(max_results/max_users)+1):

    # Inputs
    count = 0 # Counting followers per time period
    max_count = 500 # Max followers per time period
    flag = True
    next_token = params['next_token']
    

    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url()
        params = get_params()
        json_response = connect_to_endpoint(url, params)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            params['next_token'] = json_response['meta']['next_token']
            
            print("Next Token: ", params['next_token'])
            if result_count is not None and result_count > 0 and params['next_token'] is not None:
                # print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_followers += result_count
                print("Total # of followers added: ", total_followers)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                # print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_followers += result_count
                print("Total # of followers added: ", total_followers)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_followers)

-------------------
Token:  None
200
Next Token:  MTN0C90S1GFHCZZZ
# of Followers added from this response:  100
Total # of followers added:  100
-------------------
-------------------
Token:  None
429


Exception: Request returned an error: 429 Rate limit exceeded
